In [1]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import numpy as np
import pickle as pkl
import seaborn as sns

from IPython.core.interactiveshell import InteractiveShell
from functools import partial, reduce
from datetime import datetime

InteractiveShell.ast_node_interactivity = "all"

os.chdir('/mnt/data1/songziwu/data/crc_data/')

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [3]:
preIM_path = '/mnt/data1/songziwu/data/crc_data/01_MQ_Incident_Match_Files/'
preIMcontrols = pd.read_csv(preIM_path + "preIMcontrols.csv")
preIMcasesCRC = pd.read_csv(preIM_path + "preIMcasesCRC.csv")
preIMcasesCC = pd.read_csv(preIM_path + "preIMcasesCC.csv")
preIMcasesRC = pd.read_csv(preIM_path + "preIMcasesRC.csv")

In [4]:
def pkl_dump(data,file):
    with open(file,"wb") as fw:
        pkl.dump(data,fw)
        
def pkl_load(file):
    with open(file,"rb") as fr:
        data = pkl.load(fr)
    return data


In [5]:
# function of cross table frequency and column percentage
def crosstable_oc(df,row,column,round=4):
    crosstable = pd.crosstab(index=df[row],columns = df[column], margins = True)
    crosstable.index.values[-1] = 'coltotal'
    #column percentage
    col_p = np.round(crosstable / crosstable.loc["coltotal"],round)*100
    
    
    print('\n')
    print('Frequency')
    print(crosstable)
    print('\n')
    print('Column%')
    print(col_p)
    
# ADD CRC STATUS FILE
def addCRCst(df,otc):
    df = df.merge(otc,left_on='PATID',right_on='PATID')
    df.rename(columns={'Outcome':'CRC_STATUS'},inplace=True)
    return df

## Load Data

In [6]:
df_outcome = pd.read_csv('Outcome.csv', dtype = str)
print(df_outcome.shape)


(35785, 2)


In [7]:
df_enc = pd.read_csv('ENCOUNTER.csv', dtype = str)
df_enc = addCRCst(df_enc,df_outcome)
print(df_enc.shape)

(7964989, 34)


In [8]:
df_enc.head()

,ENCOUNTERID,PATID,ADMIT_DATE,ADMIT_TIME,DISCHARGE_DATE,DISCHARGE_TIME,PROVIDERID,FACILITY_LOCATION,ENC_TYPE,FACILITYID,DISCHARGE_DISPOSITION,DISCHARGE_STATUS,DRG,DRG_TYPE,ADMITTING_SOURCE,PAYER_TYPE_PRIMARY,PAYER_TYPE_SECONDARY,FACILITY_TYPE,RAW_SITEID,RAW_ENC_TYPE,RAW_DISCHARGE_DISPOSITION,RAW_DISCHARGE_STATUS,RAW_DRG_TYPE,RAW_ADMITTING_SOURCE,RAW_FACILITY_TYPE,RAW_PAYER_TYPE_PRIMARY,RAW_PAYER_NAME_PRIMARY,RAW_PAYER_ID_PRIMARY,RAW_PAYER_TYPE_SECONDARY,RAW_PAYER_NAME_SECONDARY,RAW_PAYER_ID_SECONDARY,UPDATED,SOURCE,CRC_STATUS
0,a+xOB8oBTFCu3wa/ePVtJjR9uvkS+jd1znFMUN+oBM0J8x...,11e75061a5e8c7c8a3fa0050569ea8fb,2013-04-19,08:15,2013-04-25,13:43,a+xOc7gHSlar,327,IP,G2,A,HO,885,01,OT,2,81,HOSPITAL_PSYCHIATRIC,NaN,NaN,1,01,1213 CMS DRG LIST,1,G2,MEDICAID,MEDICAID OF FLORIDA,20010,SELF PAY,COMM CARE PENDING FOLLOW-UP,01084,2018-09-25 10:49:07.000,ORL,0
1,a+xOAs1zOVWg2XTKDfkWKDcOzPxihz5zsXc9X66vDLkNgm...,11e75061a5e8c7c8a3fa0050569ea8fb,2014-12-23,20:39,2014-12-24,04:24,a+xOc7gCQlCh,328,ED,A1,A,HO,000,NaN,OT,511,81,OT,NaN,NaN,1,01,NaN,1,A1,HMO,SIMPLY CLEAR HEALTH ALLIANCE,20084,SELF PAY,SP AFTER INSURANCE,01004,2018-09-25 10:50:47.000,ORL,0
2,a+xOArgGPVfb2XC5CfFtVDJ7yIoR/DRxzgY4JKiicL1y9h...,11e75061a5e8c7c8a3fa0050569ea8fb,2014-08-27,19:40,2014-08-27,22:28,a+xOc7gDQ1eq,328,ED,A1,A,HO,000,NaN,OT,511,81,OT,NaN,NaN,1,01,NaN,1,A1,HMO,SIMPLY CLEAR HEALTH ALLIANCE,20084,SELF PAY,SP AFTER INSURANCE,01004,2018-09-25 10:50:29.000,ORL,0
3,a+xOAbAKQ1SqqHS9D4cfVjIPyopiiUcGugY5ItivDboP9x...,11e75061a5e8c7c8a3fa0050569ea8fb,2012-06-03,12:25,2012-06-03,15:30,a+xOc7gDTFev,328,ED,A1,A,HO,000,NaN,OT,2,81,OT,NaN,NaN,1,01,NaN,1,A1,MEDICAID,ZMEDIPASS STATE MEDICAID HMO,20012,SELF PAY,SP AFTER INSURANCE,01004,2018-09-25 10:48:04.000,ORL,0
4,a+xOc74KTyesowa5D/MdI0J/yP9ijzB6uwc6J9qpd8oPgx...,11e75061a5e8c7c8a3fa0050569ea8fb,2015-05-31,03:23,2015-06-01,13:04,a+xOc7gHT1Sr,327,IP,G2,A,HO,885,01,OT,511,81,HOSPITAL_PSYCHIATRIC,NaN,NaN,1,01,1415 CMS DRG LIST,1,G2,HMO,BEH SIMPLY CHA BEACON HS MCAID,20102,SELF PAY,SP AFTER INSURANCE,01004,2018-09-25 10:51:19.000,ORL,0


In [9]:
df_diag = pd.read_csv('DIAGNOSIS.csv',dtype = str)
df_diag = addCRCst(df_diag,df_outcome)
print(df_diag.shape)

(16212826, 22)


In [10]:
df_diag['DX'] = df_diag['DX'].apply(lambda x: str.replace(x, ".", ""))
df_diag['DX3'] = df_diag.DX.apply(lambda x: x[:3])
df_diag['DX4'] = df_diag.DX.apply(lambda x: x[:4])
df_diag['DX5'] = df_diag.DX.apply(lambda x: x[:5])
df_diag['DX6'] = df_diag.DX.apply(lambda x: x[:6])

In [11]:
df_icd9 = df_diag[df_diag['DX_TYPE'].isin(['09'])]
df_icd10 = df_diag[df_diag['DX_TYPE'].isin(['10'])]

In [12]:
def extract_history(df_diag_pt, criteriavar, idvar, datevar, returnvar, extractname):
    
    # format the date
    df_diag_pt['ADMIT_DATE'] = pd.to_datetime(df_diag_pt['ADMIT_DATE'])
    
#     # find at least 2 outpatient encounters within a year or at least 1 inpatient encounter 
#     inpatient = _df_meetcriteria[_df_meetcriteria['ENC_TYPE'].isin(['EI', 'IP', 'IS', 'IC'])]
#     outpatient = _df_meetcriteria[_df_meetcriteria['ENC_TYPE'].isin(['AV', 'OA', 'OS'])]

#     outpatient['ADMIT_DATE'] = outpatient['ADMIT_DATE'].astype(str)
#     outpat_datetime = outpatient.sort_values(['ADMIT_DATE'],ascending = True).groupby('PATID').agg({'ADMIT_DATE': lambda x: ','.join(x)}).reset_index()
#     outpat_datetime['DATES_LIST'] = outpat_datetime.ADMIT_DATE.apply(lambda x: x.split(','))
#     outpat_datetime['DATES_LIST'] = outpat_datetime['DATES_LIST'].apply(lambda x: list(set(x)))
#     outpat_datetime['DIFF'] = outpat_datetime.DATES_LIST.apply(lambda x: minimum_absolute_difference(x))

#     # patids with at least 2 outpatient encounters within a year or at least 1 inpatient encounter 
#     final_patient_id = pd.concat([inpatient[['PATID']], outpat_datetime[(outpat_datetime['DIFF']>0) & (outpat_datetime['DIFF'] <= 365)][['PATID']]]).drop_duplicates()
    
#     # merge with _df_meetcriteria
#     _df_meetcriteria = _df_meetcriteria.merge(final_patient_id, left_on = 'PATID', right_on = 'PATID', how = 'inner')

    # create df with unique patient
    _df_uniquepatid = pd.DataFrame({idvar: df_diag_pt[idvar].unique()})
    # create indicator of ever has the criteriacode
    _df_uniquepatid['has_'+extractname] = _df_uniquepatid[idvar].isin(df_diag_pt[idvar]).astype(int)

    # obtain the last record for each patient with criteria code
    _df_firstrecord = _extract_first_record(df_diag_pt, idvar, datevar, returnvar, extractname)

    # merge the tables
    _df_list = [_df_uniquepatid, _df_firstrecord]
    _df_out = reduce(lambda left, right: pd.merge(left, right, on=[idvar], how='outer'), _df_list)
    return _df_out

def _extract_2nd_record(df_long, idvar, datevar, returnvar, extractname):
    ''' this function return the last date for each idvar for long format df '''
    _df_2 = df_long.groupby(idvar)[datevar].apply(lambda x: x.drop_duplicates().nsmallest(2)).reset_index()
    idx2 = _df_2.level_1.loc[_df_2.groupby(idvar)[datevar].idxmax()]                     
    mask2 = df_long.index.isin(idx2)
    _df_2 = df_long[mask2]
    _df_2 = _df_2[[idvar,datevar,returnvar]]
    _df_2.rename(columns={datevar:extractname + '_2nd_date', returnvar:extractname + '_2nd_code'}, inplace=True)
    return _df_2

def _extract_first_record(df_long, idvar, datevar, returnvar, extractname):
    ''' this function return the last date for each idvar for long format df '''
    _df_f = df_long.loc[df_long.groupby(idvar)[datevar].idxmin(), [idvar,datevar,returnvar]]
    _df_f.rename(columns={datevar:extractname + '_f_date', returnvar:extractname + '_f_code'}, inplace=True)
    return _df_f

def get_patids(df_icd9,df_icd10,icd9_3,icd9_4,icd9_5,icd9_6,icd10_3,icd10_4,icd10_5,icd10_6):
    
    df_icd9 = df_icd9[df_icd9['DX3'].isin(icd9_3) | df_icd9['DX4'].isin(icd9_4) | df_icd9['DX5'].isin(icd9_5) | df_icd9['DX6'].isin(icd9_6)]
    df_icd10 = df_icd10[df_icd10['DX3'].isin(icd10_3) | df_icd10['DX4'].isin(icd10_4) | df_icd10['DX5'].isin(icd10_5) | df_icd10['DX6'].isin(icd10_6)]
    df_result = pd.concat([df_icd9, df_icd10])
    
#     inpatient = df_result[df_result['ENC_TYPE'].isin(['EI', 'IP', 'IS', 'IC'])]
#     outpatient = df_result[df_result['ENC_TYPE'].isin(['AV', 'OA', 'OS'])]
    
#     outpatient['ADMIT_DATE'] = outpatient['ADMIT_DATE'].astype(str)
#     outpat_datetime = outpatient.sort_values(['ADMIT_DATE'],ascending = True).groupby('PATID').agg({'ADMIT_DATE': lambda x: ','.join(x)}).reset_index()
#     outpat_datetime['DATES_LIST'] = outpat_datetime.ADMIT_DATE.apply(lambda x: x.split(','))
#     # remove encounters on same day
#     outpat_datetime['DATES_LIST'] = outpat_datetime['DATES_LIST'].apply(lambda x: list(set(x)))
#     outpat_datetime['DIFF'] = outpat_datetime.DATES_LIST.apply(lambda x: minimum_absolute_difference(x))
    
#     final_patient = pd.concat([inpatient[['PATID']], outpat_datetime[(outpat_datetime['DIFF']>0) & (outpat_datetime['DIFF'] <= 365)][['PATID']]]).drop_duplicates()

    return df_result

def minimum_absolute_difference(dates_list):
    
    integers_list = [datetime.strptime(d, "%Y-%m-%d") for d in dates_list]
    
    min_absolute_difference = float("inf")
    
    sorted_integers_list = sorted(integers_list)
    
    for index in range(0, len(sorted_integers_list)-1):
        small_num = sorted_integers_list[index]
        large_num = sorted_integers_list[index+1]
        absolute_difference = abs((large_num-small_num).days)
        if absolute_difference <= min_absolute_difference:
            min_absolute_difference = absolute_difference
    
    # we have the exact min_absolute_difference
    min_pairs_list = []
    
    for index in range(0, len(sorted_integers_list)-1):
        small_num = sorted_integers_list[index]
        large_num = sorted_integers_list[index+1]
        absolute_difference = abs((large_num-small_num).days)
        if absolute_difference == min_absolute_difference:
            min_pairs_list.append([small_num, large_num])
    return min_absolute_difference

In [13]:
# CRC_ICD_CODES = ['C180','C181','C182','C183','C184','C185','C186',
#           'C187','C188','C189','1530','1531','1532','1533','1534',
#           '1535','1536','1537','1538','1539','Z85038','D010',
#           '2303','V1005','C19','C20','1540','1541','1548','D012','D011',
#           '2304','Z85048','V1006']

CC_ICD9_3 = []
CC_ICD9_4 = ['1530','1531','1532','1533','1534','1535','1536','1537','1538','1539','2303']
CC_ICD9_5 = ['V1005']
CC_ICD9_6 = []

CC_ICD10_3 = []
CC_ICD10_4 = ['C180','C181','C182','C183','C184','C185','C186','C187','C188','C189','D010']
CC_ICD10_5 = []
CC_ICD10_6 = ['Z85038']

RC_ICD9_3 = []
RC_ICD9_4 = ['1540','1541','1548','2304']
RC_ICD9_5 = ['V1006']
RC_ICD9_6 = []

RC_ICD10_3 = ['C19','C20']
RC_ICD10_4 = ['D012','D011']
RC_ICD10_5 = []
RC_ICD10_6 = ['Z85048']

In [14]:
df_CC_pt = get_patids(df_icd9, df_icd10, 
             CC_ICD9_3, CC_ICD9_4, CC_ICD9_5, CC_ICD9_6,
             CC_ICD10_3, CC_ICD10_4, CC_ICD10_5, CC_ICD10_6)

df_RC_pt = get_patids(df_icd9, df_icd10, 
             RC_ICD9_3, RC_ICD9_4, RC_ICD9_5, RC_ICD9_6,
             RC_ICD10_3, RC_ICD10_4, RC_ICD10_5, RC_ICD10_6)

In [15]:
print(df_CC_pt['PATID'].nunique())
print(df_RC_pt['PATID'].nunique())

1214
584


In [16]:
# all CRC diagnoses
# df_CRC = extract_history(df_diag, CRC_ICD_CODES,'DX','PATID','ADMIT_DATE','DX','CRC')
# df_CRC.head()
# df_CRC.shape

# df_CRC.groupby('has_CRC').PATID.size()
# df_CRC.groupby('CRC_f_code').PATID.size().sort_values(0,ascending=False)

# df_CRC[df_CRC.has_CRC == 1].head()

In [17]:
# all CC diagnoses
df_CC = extract_history(df_CC_pt, 'DX','PATID','ADMIT_DATE','DX','CRC')
df_CC.head()
df_CC.shape

df_CC.groupby('has_CRC').PATID.size()
df_CC.groupby('CRC_f_code').PATID.size().sort_values(0,ascending=False)

df_CC[df_CC.has_CRC == 1].head()

,PATID,has_CRC,CRC_f_date,CRC_f_code
0,11e75061126eaa76a7850050569ea8fb,1,2015-05-26,1539
1,11e750610d398f62929b0050569ea8fb,1,2014-12-30,1539
2,11e75060b0b8ccf8ab480050569ea8fb,1,2015-06-09,1535
3,11e7506128a92532845d0050569ea8fb,1,2015-06-11,V1005
4,11e750618b7669a4ad370050569ea8fb,1,2014-02-25,1536


(1214, 4)

has_CRC
1    1214
Name: PATID, dtype: int64

CRC_f_code
C189      386
1539      163
Z85038    138
C187      126
V1005      88
C181       51
C182       47
C180       35
C184       32
C186       29
1535       18
C183       16
1534       14
1533       14
D010       10
C188        9
1531        8
1536        7
C185        6
2303        5
1530        5
1538        3
1532        3
1537        1
Name: PATID, dtype: int64

,PATID,has_CRC,CRC_f_date,CRC_f_code
0,11e75061126eaa76a7850050569ea8fb,1,2015-05-26,1539
1,11e750610d398f62929b0050569ea8fb,1,2014-12-30,1539
2,11e75060b0b8ccf8ab480050569ea8fb,1,2015-06-09,1535
3,11e7506128a92532845d0050569ea8fb,1,2015-06-11,V1005
4,11e750618b7669a4ad370050569ea8fb,1,2014-02-25,1536


In [18]:
# all RC diagnoses
df_RC = extract_history(df_RC_pt,'DX','PATID','ADMIT_DATE','DX','CRC')
df_RC.head()
df_RC.shape

df_RC.groupby('has_CRC').PATID.size()
df_RC.groupby('CRC_f_code').PATID.size().sort_values(0,ascending=False)

df_RC[df_RC.has_CRC == 1].head()

,PATID,has_CRC,CRC_f_date,CRC_f_code
0,11e75060edac71e6a7850050569ea8fb,1,2015-03-27,1540
1,11e7506189545172ab480050569ea8fb,1,2014-06-19,1541
2,11e7506099fd65328d0b0050569ea8fb,1,2014-09-26,1540
3,11e750618b7669a4ad370050569ea8fb,1,2014-03-10,1540
4,11e75060c34cc98cbb6a0050569ea8fb,1,2014-02-24,2304


(584, 4)

has_CRC
1    584
Name: PATID, dtype: int64

CRC_f_code
C20       248
C19       120
Z85048     73
1541       61
1540       43
V1006      29
1548        4
D012        3
2304        2
D011        1
Name: PATID, dtype: int64

,PATID,has_CRC,CRC_f_date,CRC_f_code
0,11e75060edac71e6a7850050569ea8fb,1,2015-03-27,1540
1,11e7506189545172ab480050569ea8fb,1,2014-06-19,1541
2,11e7506099fd65328d0b0050569ea8fb,1,2014-09-26,1540
3,11e750618b7669a4ad370050569ea8fb,1,2014-03-10,1540
4,11e75060c34cc98cbb6a0050569ea8fb,1,2014-02-24,2304
